In [2]:
import docx
from docx import Document
from docx.text.paragraph import Paragraph
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.table import Table
import itertools
import re
import os
import pandas as pd
import fitz
import pdfplumber

from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.messages import HumanMessage
from langchain_openai import AzureOpenAI, AzureChatOpenAI
from langchain_openai.embeddings import AzureOpenAIEmbeddings
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [3]:
ref_file_path = r"C:\Users\320271965\Desktop\project 1 Traceability\BI\Data\DHF345094_RequirementsTraceabilityMatrix_UNS_SRS_AzurionR3 0 Rev A.xlsx"

uns_file_path = r"C:\Users\320271965\Desktop\project 1 Traceability\BI\Data\DHF335296_Azurion R3.0_Puma_UNS.docx"

srs_file_path = r"C:\Users\320271965\Desktop\project 1 Traceability\BI\Data\DHF334523_SRS_Azurion_R3.0_RevF.docx"

In [4]:
df_ref = pd.read_excel(ref_file_path, sheet_name='Traceability Matrix', engine='openpyxl')
df_ref_grouped = df_ref.groupby('User Needs Specification Tag')['System Requirements Specification Tag'].apply(lambda x: list(set(x))).reset_index()
output_file_path = r'C:\Users\320271965\Desktop\project 1 Traceability\BI\Data\df_traceability_matrix.xlsx'
df_ref_grouped.to_excel(output_file_path, index=False)
print(f"Grouped data has been saved to {output_file_path}")
df_ref_grouped


Grouped data has been saved to C:\Users\320271965\Desktop\project 1 Traceability\BI\Data\df_traceability_matrix.xlsx


,User Needs Specification Tag,System Requirements Specification Tag
0,"Not applicable, no user requirement","[SRS.Allura.Manf.VerifySystem, SRS.Allura.Phys..."
1,UNS.SmartSuite.2DQAAccuracyAndPrecision,"[SRS.Allura.Func.VesselAnalysis.Accuracy, SRS...."
2,UNS.SmartSuite.2DQAAutomatedMeasurement,"[SRS.Allura.Func.VesselAnalysis, SRS.Allura.Fu..."
3,UNS.SmartSuite.2DQACalibration,[SRS.Allura.Func.QACalibration]
4,UNS.SmartSuite.2DQAEfficientToUse,"[SRS.Allura.Func.VesselAnalysis, SRS.Allura.Fu..."
...,...,...
181,UNS.SmartSuite.ViewingROI,"[SRS.Allura.Func.AngulateBeam, SRS.Allura.Func..."
182,UNS.SmartSuite.Workspot,[SRS.Allura.Intgr.Interfaces.RealTimeImageLink...
183,UNS.SmartSuite.X-rayBeamShaping,"[SRS.Allura.Func.Move/ResetWedges, SRS.Allura...."
184,UNS.SmartSuite.X-rayDetectorFieldSize,"[SRS.Allura.Conf.SupportedConfigurations, SRS...."


In [5]:

system_requirements_spec = df_ref_grouped['System Requirements Specification Tag']

# Display the column values
system_requirements_spec


0      [SRS.Allura.Manf.VerifySystem, SRS.Allura.Phys...
1      [SRS.Allura.Func.VesselAnalysis.Accuracy, SRS....
2      [SRS.Allura.Func.VesselAnalysis, SRS.Allura.Fu...
3                        [SRS.Allura.Func.QACalibration]
4      [SRS.Allura.Func.VesselAnalysis, SRS.Allura.Fu...
                             ...                        
181    [SRS.Allura.Func.AngulateBeam, SRS.Allura.Func...
182    [SRS.Allura.Intgr.Interfaces.RealTimeImageLink...
183    [SRS.Allura.Func.Move/ResetWedges, SRS.Allura....
184    [SRS.Allura.Conf.SupportedConfigurations, SRS....
185              [SRS.Allura.Func.SelectProcessingFocus]
Name: System Requirements Specification Tag, Length: 186, dtype: object

In [14]:
user_needs_spec = df_ref_grouped['User Needs Specification Tag']

# Display the column values
user_needs_spec 

0          Not applicable, no user requirement
1      UNS.SmartSuite.2DQAAccuracyAndPrecision
2      UNS.SmartSuite.2DQAAutomatedMeasurement
3               UNS.SmartSuite.2DQACalibration
4            UNS.SmartSuite.2DQAEfficientToUse
                        ...                   
181                  UNS.SmartSuite.ViewingROI
182                    UNS.SmartSuite.Workspot
183            UNS.SmartSuite.X-rayBeamShaping
184      UNS.SmartSuite.X-rayDetectorFieldSize
185             UNS.SmartSuite.processingfocus
Name: User Needs Specification Tag, Length: 186, dtype: object

In [7]:
file_path=r'C:\Users\320271965\Desktop\project 1 Traceability\BI\IGT_BI\srs_content_nd_tags.xlsx'
df=pd.read_excel(file_path)
srs_data = []
for srs_record in system_requirements_spec:
    #print(srs_record)
    extracted_content=""
    for srs_tag in srs_record:
        print(srs_tag)
        result=df[df.iloc[:,0]==srs_tag]
        if not result.empty:
            content = result.iloc[0, 1]
            if pd.isna(content):
                content = ""  
            else:
                content = str(content)  
            
            extracted_content += content        
    srs_data.append([srs_record, extracted_content])
    #print(data)    
output_df2 = pd.DataFrame(srs_data, columns=['srs_record', 'extracted_content']) 
output_file_path = r'C:\Users\320271965\Desktop\project 1 Traceability\BI\IGT_BI\traceability with srs content.xlsx'
output_df2.to_excel(output_file_path, index=False)

print(f"Data has been successfully written to {output_file_path}")   


SRS.Allura.Manf.VerifySystem
SRS.Allura.I&V.Environment
SRS.Allura.Phys.TechnicalRoomCabinetsSpace
SRS.Allura.I&V.TestInterface
SRS.Allura.SLS.ECO.EnergyConsumption
SRS.Allura.Manf.LoadConfiguration
SRS.Allura.I&V.ManualTest
SRS.Allura.Func.VentricularAnalysisAccuracy
SRS.Allura.Func.VesselAnalysis.Accuracy
SRS.Allura.Func.VesselAnalysis
SRS.Allura.Func.VentricularAnalysis
SRS.Allura.Func.QA
SRS.Allura.Func.QACalibration
SRS.Allura.Func.VesselAnalysis
SRS.Allura.Func.VentricularAnalysis
SRS.Allura.Func.QA
SRS.Allura.Func.DualPhaseXperCT
SRS.Allura.Func.ClassicDRA
SRS.Allura.Func.DRAFor3D
SRS.Allura.Func.XperCT
SRS.Allura.Func.DualPhaseXperCT
SRS.Allura.Func.ClassicDRA
SRS.Allura.Func.DRAFor3D
SRS.Allura.Func.XperCT
SRS.Allura.Func.DisplayFluoroTime
SRS.Allura.Conf.ImageStorageOptions
SRS.Allura.Func.ChangeFluoroProcessing
SRS.Allura.Qual.IQContrastResolution2D
SRS.Allura.Func.ActivateExposure
SRS.Allura.Func.DoseControlTechniques
SRS.Allura.Func.SelectDetectorFieldsize
SRS.Allura.Qual.

In [10]:
doc_path = r'C:\Users\320271965\Desktop\project 1 Traceability\BI\Data\DHF334523_SRS_Azurion_R3.0_RevF.docx'
srs_doc = docx.Document(doc_path)

# Provided function to extract content under each tag
def extract_content_and_tables_after_tag(srs_doc, srs_tag):
    elements = []  # List to hold text and tables in order
    tag_found = False
    processing_content = False
    in_exclusion_section = False  # Variable to handle multiple exclusion sections
    content_found = False
    table_found = False

    def starts_with_srs_tag(text, tag_start):
        """Check if text starts with a specific tag."""
        return text.startswith(tag_start)

    def is_another_srs_tag(text):
        return text.startswith('SRS.')

    def is_heading(text):
        """Check if text is a heading."""
        return text.startswith('Heading')

    # Iterate through elements (paragraphs and tables) in the document body
    for element in srs_doc.element.body:
        if isinstance(element, docx.oxml.CT_P):  # Check for paragraphs
            para = docx.text.paragraph.Paragraph(element, srs_doc)
            para_text = para.text

            # Check if we are in an exclusion section
            if 'LIST OF REQUIREMENT TAGS' in para_text or 'Revision History' in para_text:
                in_exclusion_section = True
            elif para.style.name.startswith('Heading') and in_exclusion_section:
                in_exclusion_section = False

            if not in_exclusion_section:
                if starts_with_srs_tag(para_text, srs_tag):
                    if tag_found:  # If another uid_tag is found, stop
                        break
                    tag_found = True
                    processing_content = True
                    continue

                if processing_content:
                    if is_another_srs_tag(para_text) or is_heading(para_text):
                        break  # Stop if another uid_tag is found
                    elements.append(('text', para_text))
                    content_found = True

        elif isinstance(element, docx.oxml.CT_Tbl):  # Check for tables
            if processing_content:
                table = docx.table.Table(element, srs_doc)
                table_data = [[cell.text for cell in row.cells] for row in table.rows]
                elements.append(('table', table_data))
                table_found = True

    return elements if content_found or table_found else []

# Convert extracted content into a format suitable for Excel
output_data = []

for tag_list in system_requirements_spec:
    if isinstance(tag_list, list):  # Ensure it's a list
        combined_content = ""  # To hold combined content for all tags in the current tag_list

        # Process each tag in the current tag_list
        for tag in tag_list:
            elements = extract_content_and_tables_after_tag(srs_doc, tag)
            
            # Combine content for this tag
            for element_type, data in elements:
                if element_type == "text":
                    combined_content += data + "\n"
                elif element_type == "table":
                    table_str = "\n".join(["\t".join(row) for row in data])
                    combined_content += f"Table:\n{table_str}\n"

        # Append the combined content for the current tag_list
        output_data.append([" | ".join(tag_list), combined_content.strip()])

# Save the output to an Excel file
output_df = pd.DataFrame(output_data, columns=['SRS Tags (Combined)', 'Content'])
output_df.to_excel(r'C:\Users\320271965\Desktop\project 1 Traceability\BI\Data\traceability_with_combinedsrs_content.xlsx', index=False)

print("Content for each tag list has been saved.")

# Save the output to an Excel file
output_df = pd.DataFrame(output_data, columns=['SRS Tags (Combined)', 'Content'])
output_df.to_excel(r'C:\Users\320271965\Desktop\project 1 Traceability\BI\Data\traceability with srs content.xlsx', index=False)

print("Content under each tag, including tables, has been saved.")


Content for each tag list has been saved.
Content under each tag, including tables, has been saved.


In [ ]:
file_path=r'C:\Users\320271965\Desktop\project 1 Traceability\BI\IGT_BI\processed_uns_tags_with_tables.xlsx'
df=pd.read_excel(file_path)
uns_data = []

for uns_tag in user_needs_spec:
    extracted_content=""
    print(uns_tag)
    result=df[df.iloc[:,0]==uns_tag]
    if not result.empty:
        extracted_content=result.iloc[0,1]
    uns_data.append([uns_tag, extracted_content])    
    #print(extracted_content) 
output_df1 = pd.DataFrame(uns_data, columns=['uns_tag', 'extracted_content'])    
output_file_path = r'C:\Users\320271965\Desktop\project 1 Traceability\BI\IGT_BI\traceability with uns content.xlsx'
output_df1.to_excel(output_file_path, index=False)

print(f"Data has been successfully written to {output_file_path}") 
              
    
    


Not applicable, no user requirement
UNS.SmartSuite.2DQAAccuracyAndPrecision
UNS.SmartSuite.2DQAAutomatedMeasurement
UNS.SmartSuite.2DQACalibration
UNS.SmartSuite.2DQAEfficientToUse
UNS.SmartSuite.3DAnatomyScan
UNS.SmartSuite.3Dacquisition
UNS.SmartSuite.AcquisitionImageQualityAndDose
UNS.SmartSuite.ActivateFluoroExposure
UNS.SmartSuite.AddPatientData
UNS.SmartSuite.AdditionalFlexSpot
UNS.SmartSuite.Administration
UNS.SmartSuite.AmbientVideos
UNS.SmartSuite.Annotations
UNS.SmartSuite.AuthorizedUser
UNS.SmartSuite.AutomaticPositionControl
UNS.SmartSuite.AuxiliarySystems
UNS.SmartSuite.BiplaneContrast
UNS.SmartSuite.BiplaneFocusSelection
UNS.SmartSuite.BiplaneHeadSpace
UNS.SmartSuite.BiplaneRealTime
UNS.SmartSuite.BolusChaseReconstruction
UNS.SmartSuite.CardiacSwing
UNS.SmartSuite.Carriage
UNS.SmartSuite.CeilingCompatibility
UNS.SmartSuite.ChangeDetectorOrientation
UNS.SmartSuite.ChangeFluoroExposure
UNS.SmartSuite.Cleaning
UNS.SmartSuite.ClinicalImagesOnTSM
UNS.SmartSuite.CollisionDetect

In [13]:
uns_df1=pd.read_excel(r'C:\Users\320271965\Desktop\project 1 Traceability\BI\IGT_BI\traceability with uns content.xlsx')
srs_df1=pd.read_excel(r'C:\Users\320271965\Desktop\project 1 Traceability\BI\Data\traceability with srs content.xlsx')

In [14]:
# Concatenating horizontally (adding columns)
df_combined = pd.concat([uns_df1, srs_df1], axis=1)
print(df_combined)
output_file_path = r'C:\Users\320271965\Desktop\project 1 Traceability\BI\IGT_BI\1911traceability with uns srs content.xlsx'
df_combined.to_excel(output_file_path, index=False)

print(f"Data has been successfully written to {output_file_path}") 
              


                                     uns_tag  \
0        Not applicable, no user requirement   
1    UNS.SmartSuite.2DQAAccuracyAndPrecision   
2    UNS.SmartSuite.2DQAAutomatedMeasurement   
3             UNS.SmartSuite.2DQACalibration   
4          UNS.SmartSuite.2DQAEfficientToUse   
..                                       ...   
181                UNS.SmartSuite.ViewingROI   
182                  UNS.SmartSuite.Workspot   
183          UNS.SmartSuite.X-rayBeamShaping   
184    UNS.SmartSuite.X-rayDetectorFieldSize   
185           UNS.SmartSuite.processingfocus   

                                     extracted_content  \
0                                                  NaN   
1    Clinical images, Systematic error \n[mm], Rand...   
2    \nHave automatic contour detection in 2DQCA an...   
3    \nAutomatic and manual calibrate to convert di...   
4    \nReceive automated assistance when execute ti...   
..                                                 ...   
181  \nView from 

In [15]:
AZURE_OPENAI_API_KEY_3: 'f012e5450dfe45cb973d1b6341584c06'
AZURE_OPENAI_ENDPOINT_3: 'https://aicoe-open-ai-np-002.openai.azure.com/'
OPENAI_API_VERSION_3: '2023-07-01-preview'

CHAT_MODEL_3: 'gpt_35_turbo'
CHAT_DEPLOYMENT_3: 'aicoe-np-gpt35-turbo'

In [16]:
os.environ["AZURE_OPENAI_API_KEY_3"] = 'f012e5450dfe45cb973d1b6341584c06'
os.environ["AZURE_OPENAI_ENDPOINT_3"] = 'https://aicoe-open-ai-np-002.openai.azure.com/'
os.environ["OPENAI_API_VERSION_3"] = '2023-07-01-preview'

In [17]:
def initialize_chat_model(endpoint, api_key, deployment_name, version_num, model_name):
    return AzureChatOpenAI(azure_endpoint=endpoint, openai_api_key=api_key, 
                           deployment_name=deployment_name, openai_api_version=version_num,
                           model_name=model_name, temperature=0.0, max_tokens=2048)

In [18]:
model3 = initialize_chat_model('https://aicoe-open-ai-np-002.openai.azure.com/',
                              'f012e5450dfe45cb973d1b6341584c06',
                              'aicoe-np-gpt35-turbo', 
                              '2023-07-01-preview', 
                              'gpt_35_turbo')

In [24]:
def get_completion(prompt, model=model3,max_tokens=500): 
    message = HumanMessage(content=prompt)
    response = model.invoke([message])
    # response = model([message])
    return response.content



#def get_completion(prompt, model):
    message = HumanMessage(content=prompt)
    response = model.invoke([message])
    return response.content

In [20]:
def extract_values(response):
    is_same_match = re.search(r"is_same: (YES|NO)", response)
    is_same = is_same_match.group(1) if is_same_match else "ERROR: is_same not found"

    deviation_match = re.search(r"deviation: (\d+)%", response)  # Adjusted regex for deviation
    deviation = deviation_match.group(1) if deviation_match else "ERROR: deviation not found"

    missing_content_matches = re.findall(r"- (.*)", response)  # Adjusted regex for missing content
    missing_content = missing_content_matches if missing_content_matches else "ERROR: missing_content not found"

    return is_same, deviation, missing_content

In [25]:
# Assuming df_combined is the DataFrame and row 2 corresponds to index 1 (since indexing starts from 0)
row_index=9
uns_content = df_combined.iloc[row_index, 1]  # Second column (index 1) of row 2 (index 1)
srs_content = df_combined.iloc[row_index, 3]  # Fourth column (index 3) of row 2 (index 1)

# Format the prompt by inserting the extracted values
prompt = f"""
Your task is to compare {uns_content} and {srs_content}, enclosed in triple backticks.
uns_content is requirement given by user and srs_content is system requirements converted by test engineers.

input: ```{uns_content}, {srs_content}```
output:
Please answer below questions after comparing input, in same order.
1.) Whatever is in uns_content is being covered fully in srs_content or not? 
    is_same: YES/NO
2.) By how much percentage srs_content is deviated from uns_content?
    deviation: percentage
3.) If anything is missing from srs_content which is there in uns_content, state that clearly in pointers.
    missing_content: pointers
4.) Describe the deviation in this ?     
"""
response = get_completion(prompt, model3)


print(f"UNS content: {uns_content} \n\n\n SRS content: {srs_content}")
print(response)


BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 4096 tokens. However, you requested 4576 tokens (2528 in the messages, 2048 in the completion). Please reduce the length of the messages or completion.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

In [26]:
import pandas as pd

# List to hold the responses for each row
responses = []

# Iterate over the rows in df_combined
for index, row in df_combined.iterrows():
    try:
        uns_content = row.iloc[1]  # Second column
        srs_content = row.iloc[3]  # Fourth column
        
        
        prompt = f"""
        Your task is to compare {uns_content} and {srs_content}, enclosed in triple backticks.
        uns_content is requirement given by user and srs_content is system requirements converted by test engineers.
        
        input: ```{uns_content}, {srs_content}```
        output:
        Please answer below questions after comparing input, in same order.
        1.) Whatever is in uns_content is being covered fully in srs_content or not? 
            is_same: YES/NO
        2.) By how much percentage srs_content is deviated from uns_content?
            deviation: percentage
        3.) If anything is missing from srs_content which is there in uns_content, state that clearly in pointers.
            missing_content: pointers
        4.) Describe the deviation in this 
        """
        
       
        response = get_completion(prompt, model3)  
        
        # Add the response to the list
        responses.append({
            'Row': index + 1,
            'uns_content': uns_content,
            'srs_content': srs_content,
            'Response': response
        })

    except Exception as e:
        # If there is an error, print a message and skip to the next row
        print(f"Error processing row {index + 1}: {e}")
        continue  # Skip to the next iteration


response_df = pd.DataFrame(responses)


output_file_path = r'C:\Users\320271965\Desktop\project 1 Traceability\BI\Output\LLM4response.xlsx'
response_df.to_excel(output_file_path, index=False)

print(f"All responses saved to {output_file_path}")


Error processing row 1: Error code: 400 - {'error': {'message': "This model's maximum context length is 4096 tokens. However, your messages resulted in 9268 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}
Error processing row 8: Error code: 400 - {'error': {'message': "This model's maximum context length is 4096 tokens. However, your messages resulted in 9830 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}
Error processing row 9: Error code: 400 - {'error': {'message': "This model's maximum context length is 4096 tokens. However, you requested 5774 tokens (3726 in the messages, 2048 in the completion). Please reduce the length of the messages or completion.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}
Error processing row 10: Error code: 400 - {'error': {

In [27]:
import pandas as pd
import re  # For extracting the parts of the response

# List to hold the responses for each row
responses = []

# Iterate over the rows in df_combined
for index, row in df_combined.iterrows():
    try:
        uns_content = row.iloc[1]  # Second column
        srs_content = row.iloc[3]  # Fourth column
        
        # Format the prompt with the values from the current row
        prompt = f"""
        Your task is to compare {uns_content} and {srs_content}, enclosed in triple backticks.
        uns_content is requirement given by user and srs_content is system requirements converted by test engineers.
        
        input: ```{uns_content}, {srs_content}```
        output:
        Please answer below questions after comparing input, in the same order.
        1.) Whatever is in uns_content is being covered fully in srs_content or not? 
            is_same: YES/NO
        2.) By how much percentage srs_content is deviated from uns_content?
            deviation: percentage
        3.) If anything is missing from srs_content which is there in uns_content, state that clearly in pointers.
            missing_content: pointers
        4.) Describe the deviation in this
        """
        
        # Mocked LLM response (Replace with actual response)
        response = get_completion(prompt, model3)  # Replace with actual response from LLM
        
        # Example response:
        # response = """
        # 1.) is_same: NO
        # 2.) deviation: 60%
        # 3.) missing_content: Viewer for logging and tracing, Capabilities for content scaling...
        # 4.) The srs_content is missing several requirements...
        # """
        
        # Extracting the is_same, deviation, missing_content, and deviation_description using regular expressions
        is_same = re.search(r'is_same:\s*(YES|NO)', response)
        deviation = re.search(r'deviation:\s*([\d%]+)', response)
        missing_content = re.search(r'missing_content:\s*(.*?)(?:\n|$)', response, re.DOTALL)
        deviation_description = re.search(r'4\.\)\s*(.*)', response, re.DOTALL)
        
        # Safely get the matched values, or default to 'N/A' if not found
        is_same_value = is_same.group(1) if is_same else 'N/A'
        deviation_value = deviation.group(1) if deviation else 'N/A'
        missing_content_value = missing_content.group(1).strip() if missing_content else 'N/A'
        deviation_description_value = deviation_description.group(1).strip() if deviation_description else 'N/A'
        
        # Add the response to the list with separate columns
        responses.append({
            'Row': index + 1,
            'uns_content': uns_content,
            'srs_content': srs_content,
            'is_same': is_same_value,
            'deviation': deviation_value,
            'missing_content': missing_content_value,
            'deviation_description': deviation_description_value
        })

    except Exception as e:
        # If there is an error, print a message and skip to the next row
        print(f"Error processing row {index + 1}: {e}")
        continue  # Skip to the next iteration

# Convert the responses list to a DataFrame
response_df = pd.DataFrame(responses)

# Save the DataFrame to an Excel file
output_file_path = r'C:\Users\320271965\Desktop\project 1 Traceability\BI\Output\LLM4___response.xlsx'
response_df.to_excel(output_file_path, index=False)

print(f"All responses saved to {output_file_path}")


Error processing row 1: Error code: 400 - {'error': {'message': "This model's maximum context length is 4096 tokens. However, your messages resulted in 9269 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}
Error processing row 8: Error code: 400 - {'error': {'message': "This model's maximum context length is 4096 tokens. However, your messages resulted in 9831 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}
Error processing row 9: Error code: 400 - {'error': {'message': "This model's maximum context length is 4096 tokens. However, you requested 5775 tokens (3727 in the messages, 2048 in the completion). Please reduce the length of the messages or completion.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}
Error processing row 10: Error code: 400 - {'error': {

In [8]:
print(df_combined)

NameError: name 'df_combined' is not defined

In [18]:
import pandas as pd
import re  # For extracting the parts of the response

dff_combined=read_excel(r'C:\Users\320271965\Desktop\project 1 Traceability\BI\IGT_BI\traceability with content.xlsx')

# List to hold the responses for each row
responses = []

# Iterate over the rows in df_combined
for index, row in dff_combined.iterrows():
    try:
        uns_content = row.iloc[1]  # Second column
        srs_content = row.iloc[3]  # Fourth column
        
        # Format the prompt with the values from the current row
        prompt = f"""
        Your task is to compare {uns_content} and {srs_content}, enclosed in triple backticks.
        uns_content is requirement given by user and srs_content is system requirements converted by test engineers.
        
        input: ```{uns_content}, {srs_content}```
        output:
        Please answer below questions after comparing input, in the same order.
        1.) Whatever is in uns_content is being covered fully in srs_content or not? 
            is_same: YES/NO
        2.) If anything is missing in the srs_contentwhich is there in uns_content, state that clearly in pointers.
            missing_content: pointers
        """
        
        # Mocked LLM response (Replace with actual response)
        response = get_completion(prompt, model3,max_tokens=1000)  # Replace with actual response from LLM
        
        # Example response:
        # response = """
        # 1.) is_same: NO
        # 2.) deviation: 60%
        # 3.) missing_content: Viewer for logging and tracing, Capabilities for content scaling...
        # 4.) The srs_content is missing several requirements...
        # """
        
        # Extracting the is_same, deviation, missing_content, and deviation_description using regular expressions
        is_same = re.search(r'is_same:\s*(YES|NO)', response)
        missing_content = re.search(r'missing_content:\s*(.*?)(?:\n|$)', response, re.DOTALL)
        
        # Safely get the matched values, or default to 'N/A' if not found
        is_same_value = is_same.group(1) if is_same else 'N/A'
        missing_content_value = missing_content.group(1).strip() if missing_content else 'N/A'
        
        # Add the response to the list with separate columns
        responses.append({
            'Row': index + 1,
            'uns_content': uns_content,
            'srs_content': srs_content,
            'is_same': is_same_value,
            'missing_content': missing_content_value,
        })

    except Exception as e:
        # If there is an error, print a message and skip to the next row
        print(f"Error processing row {index + 1}: {e}")
        continue  # Skip to the next iteration

# Convert the responses list to a DataFrame
response_df = pd.DataFrame(responses)

# Save the DataFrame to an Excel file
output_file_path = r'C:\Users\320271965\Desktop\project 1 Traceability\BI\Output\ver2LLM___response.xlsx'
response_df.to_excel(output_file_path, index=False)

print(f"All responses saved to {output_file_path}")


NameError: name 'read_excel' is not defined

In [21]:
import pandas as pd
import re  # For extracting the parts of the response

# Load your Excel file
dff_combined = pd.read_excel(r'C:\Users\320271965\Desktop\project 1 Traceability\BI\IGT_BI\traceability with content.xlsx')

# Function to get the LLM response
def get_completion(prompt, model, max_tokens=800):
    message = HumanMessage(content=prompt)
    response = model.invoke([message])
    return response.content

# List to hold the responses for each row
responses = []

# Iterate over the rows in df_combined
for index, row in dff_combined.iterrows():
    try:
        uns_content = row.iloc[1]  # Assuming the second column is the UNS content
        srs_content = row.iloc[3]  # Assuming the fourth column is the SRS content
        
        # Format the prompt with the values from the current row
        prompt = f"""
        Your task is to compare the content provided below, enclosed in triple backticks.
        'uns_content' is the user requirement, while 'srs_content' is the system requirement by test engineers.
        
        Input:
        ```{uns_content}```
        ```{srs_content}```
        
        Output:
        Please answer the following questions in the same order:
        1. Does 'uns_content' fully align with 'srs_content'?
           Answer with: **is_same: YES/NO**
        2. If any content is missing in 'srs_content' that is present in 'uns_content', clearly list it as pointers.
            Answer with: **missing_content:
        """
        
        # Get response from the LLM (make sure `model3` is defined)
        response = get_completion(prompt, model3, max_tokens=1000) 
        
        # Use regular expressions to extract the is_same and missing_content
        is_same = re.search(r'is_same:\s*(YES|NO)', response)
        missing_content = re.search(r'missing_content:\s*(.*?)(?:\n|$)', response, re.DOTALL)
        
        # Safely get the matched values, or default to 'N/A' if not found
        is_same_value = is_same.group(1) if is_same else 'N/A'
        missing_content_value = missing_content.group(1).strip() if missing_content else 'N/A'
        
        # Add the response to the list with separate columns
        responses.append({
            'uns_content': uns_content,
            'srs_content': srs_content,
            'uns aligns with srs': is_same_value,
            'missing_content': missing_content_value,
        })

    except Exception as e:
        # If there is an error, print a message and skip to the next row
        print(f"Error processing row {index + 1}: {e}")
        continue  # Skip to the next iteration

# Convert the responses list to a DataFrame
response_df = pd.DataFrame(responses)

# Save the DataFrame to an Excel file (append to existing file)
output_file_path = r'C:\Users\320271965\Desktop\project 1 Traceability\BI\Output\ver2LLM___response.xlsx'

response_df.to_excel(output_file_path, index=False)

print(f"All responses have been saved to {output_file_path}.")


Error processing row 8: Error code: 400 - {'error': {'message': "This model's maximum context length is 4096 tokens. However, you requested 4576 tokens (2528 in the messages, 2048 in the completion). Please reduce the length of the messages or completion.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}
Error processing row 17: Error code: 400 - {'error': {'message': "This model's maximum context length is 4096 tokens. However, you requested 5461 tokens (3413 in the messages, 2048 in the completion). Please reduce the length of the messages or completion.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}
Error processing row 42: Error code: 400 - {'error': {'message': "This model's maximum context length is 4096 tokens. However, you requested 4524 tokens (2476 in the messages, 2048 in the completion). Please reduce the length of the messages or completion.", 'type': 'invalid_request_error', 'param': 'm

In [35]:
# Assuming df_combined is the DataFrame and row 2 corresponds to index 1 (since indexing starts from 0)
def get_completion(prompt, model, max_tokens=800):
    message = HumanMessage(content=prompt)
    response = model.invoke([message])
    return response.content

row_index=4
uns_content = dff_combined.iloc[row_index, 1]  # Second column (index 1) of row 2 (index 1)
srs_content = dff_combined.iloc[row_index, 3]  # Fourth column (index 3) of row 2 (index 1)

# Format the prompt by inserting the extracted values
prompt = f"""
Your task is to compare {uns_content} and {srs_content}, enclosed in triple backticks.
uns_content is requirement given by user and srs_content is system requirements converted by test engineers.

input: ```{uns_content}, {srs_content}```
output:
Please answer below questions after comparing input, in same order.
1.) Whatever is in uns_content is its core meaning being covered fully in srs_content or not? 
    is_same: YES/NO
2.) If anything is missing in srs_content which is there in uns_content, state that.
"""
response = get_completion(prompt, model3)


print(f"UNS content: {uns_content} \n\n\n SRS content: {srs_content}")
print(response)


APIConnectionError: Connection error.

In [ ]:
import pandas as pd
from openai import AzureOpenAI
from dotenv import load_dotenv

# Function to interact with the model
def get_completion(prompt, model=model3, max_tokens=500):
    message = HumanMessage(content=prompt)
    response = model.invoke([message])
    return response.content

# Load the DataFrame (make sure your file path and sheet name are correct)
input_file_path = r'C:\Users\320271965\Desktop\project 1 Traceability\BI\Data\UID\CombinedOutputflexibleviewing.xlsx'
combined_df1 = pd.read_excel(input_file_path)

# List to store results
results = []

# Iterate through each row in the DataFrame (Limited to 150 rows)


for index, row in combined_df1.iterrows(): 

    uid_tag = row['Uid tag']  # Assuming UID tag is in this column
    uid_content = row.iloc[1]  # Adjust column index as per your data
    srs_tag = row['Srs tag']  # Assuming SRS tag is in this column
    srs_content = row.iloc[3]  # Adjust column index as per your data

    # Generate prompt
    prompt = f"""
    I have two pieces of content that need to be compared:

    UID Content:
    ```{uid_content}```

    SRS Content:
    ```{srs_content}```

    Please answer the following questions:

    1. Is there similarity between the contents?

       - Answer with: **what is similar? and what is missing or not similar?**
   
    """

    # Get response from the model
    try:
        response = get_completion(prompt)
    except Exception as e:
        response = f"Error: {e}"

    # Store the results in a dictionary
    results.append({
        'UID Tag': uid_tag,
        'UID Content': uid_content,
        'SRS Tag': srs_tag,
        'SRS Content': srs_content,
        'Comparison Result': response
    })

# Convert results to a DataFrame
output_df = pd.DataFrame(results)

output_path = r'C:\Users\320271965\Desktop\project 1 Traceability\BI\Data\Comparison_Output_content150.xlsx'
output_df.to_excel(output_path, index=False)


print(f"Comparison results for 150 rows have been saved to {output_path}.")


In [36]:
import pandas as pd

# Load the original Excel file and specify the sheet
input_file_path = r'C:\Users\320271965\Desktop\project 1 Traceability\BI\Output\LLM___response.xlsx'

# Read the sheet into a DataFrame
df = pd.read_excel(input_file_path)

# Drop the 'deviation' column
df_modified = df.drop(columns=['deviation'])

# Save the modified DataFrame to a new Excel file
output_file_path = r'C:\Users\320271965\Desktop\project 1 Traceability\BI\Output\LLM_responses_no_deviation.xlsx'
df_modified.to_excel(output_file_path, index=False)

print(f"Column 'deviation' has been dropped and saved to a new file: {output_file_path}")


Column 'deviation' has been dropped and saved to a new file: C:\Users\320271965\Desktop\project 1 Traceability\BI\Output\LLM_responses_no_deviation.xlsx
